```
cd /Users/dorislee/Desktop/Fall2016/Research/seg/crowd-seg/analysis
bash herokuDBupdate.sh
```

In [2]:
cd ../web-app/

/Users/dorislee/Desktop/Fall2016/Research/seg/crowd-seg/web-app


In [3]:
%pylab inline 
%load_ext autoreload
%autoreload 2
from analysis_toolbox import *
import ast

Populating the interactive namespace from numpy and matplotlib


In [4]:
save_db_as_csv()
img_info,object_tbl,bb_info,hit_info = load_info()

/Users/dorislee/.virtualenvs/py27dev/lib/python2.7/site-packages/pandas/io/parsers.py:648: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.
  ParserWarning)


Add a new ``status`` column to hit_info

In [4]:
hit_info['status'] = pd.Series(['undetermined' for _i in range(len(hit_info))], index=hit_info.index)

In [111]:
from boto.mturk.connection import MTurkRequestError
from boto.mturk.connection import MTurkConnection
from secret import SECRET_KEY,ACCESS_KEY
DEV_ENVIROMENT_BOOLEAN = True
#This allows us to specify whether we are pushing to the sandbox or live site.
if DEV_ENVIROMENT_BOOLEAN:
    AMAZON_HOST = 'mechanicalturk.sandbox.amazonaws.com'
    MastersQualID = '2F1KVCNHMVHV8E9PBUB2A4J79LU20F'
else:
    AMAZON_HOST = 'mechanicalturk.amazonaws.com'
    MastersQualID = '2NDP2L92HECWY8NS8H3CK0CP5L9GHO'

#Start Configuration Variables
AWS_ACCESS_KEY_ID = ACCESS_KEY
AWS_SECRET_ACCESS_KEY = SECRET_KEY

connection = MTurkConnection(aws_access_key_id=AWS_ACCESS_KEY_ID,
							 aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
							 host=AMAZON_HOST)


In [139]:
all_hits = [hit for hit in connection.get_all_hits()]
error_log = open("error.log",'w')
object_tbl['approvedBB_count'] = pd.Series(np.zeros(len(object_tbl)), index=object_tbl.index)

In [142]:
for hit in all_hits:
    assignments = connection.get_assignments(hit.HITId)
    for assignment in assignments:
        task = hit_info[hit_info.hit_id==assignment.HITId]
        if len(task)>0:
            actions = ast.literal_eval(task.actions.get_values()[0])
            numClicks = actions.count("addClick")
            try: 
                if numClicks >3:
                    hit_info = hit_info.set_value(task.index[0],'status',"approved")
                    #Add additional counts to object table 
                    object_tbl = object_tbl.set_value(task.object_id,'approvedBB_count', \
                    object_tbl._iloc[task.object_id].approvedBB_count.get_values()[0]+1)                    
                    connection.approve_assignment(assignment.AssignmentId)
                else:
                    hit_info = hit_info.set_value(task.index[0],'status',"rejected")
                    connection.reject_assignment(assignment.AssignmentId)
            except:#(MTurkRequestError):
                print "Problem with rejecting/approving: ", assignment.AssignmentId
                error_log.write(assignment.AssignmentId+'\n')
error_log.close()

Problem with rejecting/approving:  3ZAK8W07I56NAZT186Z0R766NVD0UR
Problem with rejecting/approving:  3XXU1SWE8NN3VQF321I9R6UCGSL0AF
Problem with rejecting/approving:  3G5F9DBFOQPYYEOXB4GQTL76NJNVHC
Problem with rejecting/approving:  3AUQQEL7U6LNNEI1DW95NT8P12M0VG
Problem with rejecting/approving:  3DYGAII7PM0Y6NQ48IULW7APSIIPQH
Problem with rejecting/approving:  33LKR6A5KFC8HBZU4KDQ0G3E99BT1V
Problem with rejecting/approving:  3FPRZHYEPZZJ46HYWJROPR6W4E5V35
Problem with rejecting/approving:  34Z02EIMIT58AMEC819QMBK44H60T0
Problem with rejecting/approving:  34Z02EIMIT58AMEC819QMBK44H5T0S
Problem with rejecting/approving:  31IBVUNM9TREKIXRGESQQ0VT74ZFVN


In [144]:
object_tbl.to_csv('../../data/object_tbl_mturk.csv')

In [145]:
hit_info.to_csv('../../data/hit_mturk.csv')

In [ ]:
# if assignment.HITId in list(hit_info.hit_id):
#     hit_info.hit_id==assignment.HITId

In [33]:
import numpy as np
import pandas as pd
from analysis_toolbox import *
import ast
from boto.mturk.connection import MTurkRequestError
from boto.mturk.connection import MTurkConnection
from secret import SECRET_KEY,ACCESS_KEY
DEV_ENVIROMENT_BOOLEAN = True

save_db_as_csv()
img_info,object_tbl,bb_info,hit_info = load_info()
print 'Updated DB info'


#This allows us to specify whether we are pushing to the sandbox or live site.
if DEV_ENVIROMENT_BOOLEAN:
    AMAZON_HOST = 'mechanicalturk.sandbox.amazonaws.com'
    MastersQualID = '2F1KVCNHMVHV8E9PBUB2A4J79LU20F'
else:
    AMAZON_HOST = 'mechanicalturk.amazonaws.com'
    MastersQualID = '2NDP2L92HECWY8NS8H3CK0CP5L9GHO'

#Start Configuration Variables
AWS_ACCESS_KEY_ID = ACCESS_KEY
AWS_SECRET_ACCESS_KEY = SECRET_KEY

connection = MTurkConnection(aws_access_key_id=AWS_ACCESS_KEY_ID,
                             aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                             host=AMAZON_HOST)
print 'Connected to AMT'

all_hits = [hit for hit in connection.get_all_hits()]
error_log = open("error.log",'w')
if os.path.isfile('reject.log') :
    reject_log = open("reject.log",'a')
else:
    print "here"
    reject_log = open("reject.log",'w')
    reject_log.write("AssignmentId,numClicks,BBbox,object_id,worker_id")

hit_info['status'] = pd.Series(['undetermined' for _i in range(len(hit_info))], index=hit_info.index)
object_tbl['approvedBB_count'] = pd.Series(np.zeros(len(object_tbl)), index=object_tbl.index)
for hit in all_hits:
    assignments = connection.get_assignments(hit.HITId)
    for assignment in assignments:
        task = hit_info[hit_info.hit_id==assignment.HITId]
        if len(task)>0:
            actions = ast.literal_eval(task.actions.get_values()[0])
            numClicks = actions.count("addClick")
#             try:
#                 if numClicks >=3:
#                     print 'approving ', assignment.AssignmentId
# #                     hit_info = hit_info.set_value(task.index[0],'status',"approved")
# #                     #Add additional counts to object table
# #                     object_tbl = object_tbl.set_value(task.object_id,'approvedBB_count', \
# #                     object_tbl._iloc[task.object_id].approvedBB_count.get_values()[0]+1)
# #                     connection.approve_assignment(assignment.AssignmentId)
#                 else:
            print 'Putting into rejecting list :', assignment.AssignmentId
            print int(task.worker_id)
            print int(task.object_id)
            bb = bb_info[(bb_info.worker_id==int(task.worker_id)) & (bb_info.object_id==int(task.object_id))]
            BB ='"'+ bb.x_locs.get_values()[0]+'"'
            reject_log.write('{0},{1},{2},{3},{4}\n'.format(assignment.AssignmentId,numClicks,BB,int(task.object_id),int(task.worker_id)))
            print '{0},{1},{2},{3},{4}\n'.format(assignment.AssignmentId,numClicks,BB,int(task.object_id),int(task.worker_id))
                    #hit_info = hit_info.set_value(task.index[0],'status',"rejected")
                    #connection.reject_assignment(assignment.AssignmentId)
#             except:#(MTurkRequestError):
#                 print "Problem with rejecting/approving: ", assignment.AssignmentId
#                 error_log.write(assignment.AssignmentId+'\n')
# error_log.close()
# object_tbl.to_csv('../../data/object_tbl_mturk.csv')
# hit_info.to_csv('../../data/hit_mturk.csv')


Updated DB info
Connected to AMT
Putting into rejecting list : 33LKR6A5KFC8HBZU4KDQ0G3E9DS1TS
3
8
33LKR6A5KFC8HBZU4KDQ0G3E9DS1TS,56,"[302, 264, 254, 250, 250, 250, 248, 247, 247, 253, 267, 275, 290, 312, 337, 353, 363, 411, 435, 449, 462, 470, 478, 480, 494, 503, 511, 529, 532, 529, 522, 521, 516, 514, 506, 495, 493, 485, 483, 468, 453, 428, 371, 358, 334]",8,3

Putting into rejecting list : 3MD9PLUKKJ67HUFIFZ06JBHYXAIZND
3
11
3MD9PLUKKJ67HUFIFZ06JBHYXAIZND,13,"[347, 383, 385, 380, 393, 414, 429, 429, 408, 400, 395, 394, 354]",11,3

Putting into rejecting list : 3M23Y66PO3Z29XNCLQKHL7Y9F5U6S6
3
9
3M23Y66PO3Z29XNCLQKHL7Y9F5U6S6,14,"[258, 258, 291, 303, 355, 349, 357, 363, 388, 385, 379, 386, 356]",9,3

Putting into rejecting list : 3L6L49WXW1PNO8LNQ6EUVY435UG45H
3
13
3L6L49WXW1PNO8LNQ6EUVY435UG45H,13,"[243, 181, 190, 224, 238, 247, 254, 268, 272, 299, 306, 292]",13,3

Putting into rejecting list : 3R5F3LQFV3CPZFFUGE0JY0H3NKNZOS
3
16
3R5F3LQFV3CPZFFUGE0JY0H3NKNZOS,14,"[354, 353, 353, 361

In [36]:
import datetime

In [40]:
print datetime.datetime.now()

2016-11-27 10:26:15.259324


# Count the number of objects annotated 

In [5]:
import pandas as pd
save_db_as_csv()
img_info,object_tbl,bb_info,hit_info = load_info()

In [47]:
object_tbl['BB_count'] = pd.Series(np.zeros(len(object_tbl)), index=object_tbl.index)

In [48]:
for hit in hit_info.iterrows():
    objIdx = hit[1]['object_id']-1
    object_tbl = object_tbl.set_value(objIdx,'BB_count', object_tbl._iloc[objIdx].BB_count+1)                    

In [49]:
sum(object_tbl['BB_count'])

868.0

In [67]:
objId=1

In [61]:
object_tbl.to_csv("BB_count_tbl.csv")

In [68]:
BB_count_info = pd.read_csv("BB_count_tbl.csv")

In [71]:
pwd

u'/Users/dorislee/Desktop/Fall2016/Research/seg/crowd-seg/web-app'

In [69]:
maxAssignment = 40-int(BB_count_info[BB_count_info.id ==objId]["BB_count"])

In [70]:
maxAssignment

23

In [ ]:
object_tbl[object_tbl.id ==objId]["BB_count"].get_value